In [162]:
import simpy
import folium
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [163]:
df_locations = pd.read_csv('./locations.csv')
df_locations

,name,lat,lon
0,San Diego_CA,32.7157,-117.1611
1,Austin_TX,30.2672,-97.7431
2,Denver_CO,39.7392,-104.9903
3,Seattle_WA,47.6062,-122.3321
4,Charleston_SC,32.7765,-79.9311
5,Portland_ME,43.6615,-70.2793
6,Salt Lake City_UT,40.7608,-111.8910
7,Nashville_TN,36.1627,-86.7816
8,Minneapolis_MN,44.9778,-93.2650
9,Savannah_GA,32.0780,-81.0912


In [164]:
import simpy
import numpy as np

# Given data
time_matrix = np.array([
    [0, 3, 2, 3, 5, 6, 2, 4, 4, 5],
    [3, 0, 2, 4, 3, 4, 3, 2, 3, 2],
    [2, 2, 0, 3, 4, 4, 1, 3, 2, 4],
    [3, 4, 3, 0, 6, 6, 2, 5, 3, 6],
    [5, 3, 4, 6, 0, 2, 5, 1, 3, 0],
    [6, 4, 4, 6, 2, 0, 5, 2, 3, 2],
    [2, 3, 1, 2, 5, 5, 0, 3, 2, 4],
    [4, 2, 3, 5, 1, 2, 3, 0, 2, 1],
    [4, 3, 2, 3, 3, 3, 2, 2, 0, 3],
    [5, 2, 4, 6, 0, 2, 4, 1, 3, 0]
])

time_windows = [(0, 5), (2, 13), (1, 9), (9, 11), (2, 11), (5, 10), (0, 8), (7, 12), (3, 11), (8, 14)]

num_vehicles = 4
depot_index = 0
max_time_per_vehicle = 30  # Max time each vehicle can spend in the field

class Customer:
    def __init__(self, id, ready_time, due_time):
        self.id = id
        self.ready_time = ready_time
        self.due_time = due_time

class Vehicle:
    def __init__(self, env, id, depot_index, max_time):
        self.env = env
        self.id = id
        self.current_location = depot_index
        self.route = []
        self.time_spent = 0
        self.max_time = max_time
        self.finished = False
        self.served_customers = []  # List to store served customers

    def travel_to(self, customer_index):
        travel_time = time_matrix[self.current_location][customer_index]
        yield self.env.timeout(travel_time)
        self.time_spent += travel_time
        self.current_location = customer_index

    def serve_customer(self, customer):
        service_time = 1  # Assuming a fixed service time of 1 time unit
        yield self.env.timeout(service_time)
        self.time_spent += service_time

    def run(self):
        while not self.finished:
            if not self.route:
                yield self.env.timeout(1)  # Wait for new assignments
                continue

            next_customer_index = self.route.pop(0)
            next_customer = customers[next_customer_index]
            
            # Travel to customer
            yield self.env.process(self.travel_to(next_customer_index))
            
            # Wait if arrived before ready time
            if self.env.now < next_customer.ready_time:
                wait_time = next_customer.ready_time - self.env.now
                yield self.env.timeout(wait_time)
                self.time_spent += wait_time
            
            # Serve customer
            yield self.env.process(self.serve_customer(next_customer))
            
            # Record served customer
            self.served_customers.append(next_customer.id)
            
            # Log the service
            print(f"Time {self.env.now}: Vehicle {self.id} served customer {next_customer.id}")

            # Check if the vehicle needs to return to the depot
            if not self.route or self.time_spent + time_matrix[self.current_location][depot_index] > self.max_time:
                yield self.env.process(self.travel_to(depot_index))
                print(f"Time {self.env.now}: Vehicle {self.id} returned to depot")
                self.finished = True  # Mark the vehicle as finished

def vrptw_simulation(env):
    vehicles = [Vehicle(env, i, depot_index, max_time_per_vehicle) for i in range(num_vehicles)]
    
    # Simple route assignment (you might want to implement a more sophisticated algorithm)
    for i, customer in enumerate(customers):
        if i == depot_index:
            continue
        vehicle = min(vehicles, key=lambda v: len(v.route))
        vehicle.route.append(i)
    
    for vehicle in vehicles:
        env.process(vehicle.run())

    # Run the simulation until all vehicles are finished
    while any(not v.finished for v in vehicles):
        yield env.timeout(1)  # Continue until all vehicles are finished

    # Collect results
    all_results = {v.id: v.served_customers for v in vehicles}
    return all_results

# Create customers
customers = [Customer(i, time_windows[i][0], time_windows[i][1]) for i in range(len(time_windows))]

# Run the simulation
env = simpy.Environment()
results_process = env.process(vrptw_simulation(env))
env.run()

# Collect and print results
results = results_process.value

# Print results for each vehicle
for vehicle_id, served_customers in results.items():
    print(f"Vehicle {vehicle_id} served customers: {served_customers}")


Time 3: Vehicle 1 served customer 2
Time 4: Vehicle 0 served customer 1
Time 5: Vehicle 1 served customer 6
Time 6: Vehicle 3 served customer 4
Time 7: Vehicle 1 returned to depot
Time 9: Vehicle 0 served customer 5
Time 10: Vehicle 2 served customer 3
Time 10: Vehicle 3 served customer 8
Time 12: Vehicle 0 served customer 9
Time 14: Vehicle 3 returned to depot
Time 16: Vehicle 2 served customer 7
Time 17: Vehicle 0 returned to depot
Time 20: Vehicle 2 returned to depot
Vehicle 0 served customers: [1, 5, 9]
Vehicle 1 served customers: [2, 6]
Vehicle 2 served customers: [3, 7]
Vehicle 3 served customers: [4, 8]


In [165]:
for i in results.values():
    i.append(0)
    i.insert(0, 0)
    print(i)
answer = []
for i in results.values():
    answer.append(i)
print(answer)


[0, 1, 5, 9, 0]
[0, 2, 6, 0]
[0, 3, 7, 0]
[0, 4, 8, 0]
[[0, 1, 5, 9, 0], [0, 2, 6, 0], [0, 3, 7, 0], [0, 4, 8, 0]]


In [166]:
def mapping(df,routes):
    # Remove unused routes [0,0]
    filtered_routes = [route for route in routes if len(route) != 2]
    # Map coordinates for the remaining routes
    routes_coordinates = {}
    for i, route in enumerate(filtered_routes):
        coordinates = []
        for path in route:
            coordinates.append([df.iloc[path]['lat'], df.iloc[path]['lon']])
        routes_coordinates[i] = coordinates
    
    return routes_coordinates

coordinates = mapping(df_locations,answer)
coordinates

{0: [[np.float64(32.7157), np.float64(-117.1611)],
  [np.float64(30.2672), np.float64(-97.7431)],
  [np.float64(43.6615), np.float64(-70.2793)],
  [np.float64(32.078), np.float64(-81.0912)],
  [np.float64(32.7157), np.float64(-117.1611)]],
 1: [[np.float64(32.7157), np.float64(-117.1611)],
  [np.float64(39.7392), np.float64(-104.9903)],
  [np.float64(40.7608), np.float64(-111.891)],
  [np.float64(32.7157), np.float64(-117.1611)]],
 2: [[np.float64(32.7157), np.float64(-117.1611)],
  [np.float64(47.6062), np.float64(-122.3321)],
  [np.float64(36.1627), np.float64(-86.7816)],
  [np.float64(32.7157), np.float64(-117.1611)]],
 3: [[np.float64(32.7157), np.float64(-117.1611)],
  [np.float64(32.7765), np.float64(-79.9311)],
  [np.float64(44.9778), np.float64(-93.265)],
  [np.float64(32.7157), np.float64(-117.1611)]]}

In [167]:
# Map
map_center = [df_locations['lat'].mean(), df_locations['lon'].mean()]
custom ="cartodb positron"
map = folium.Map(location=map_center, zoom_start=4, tiles=custom)

# Choose location
locations = df_locations

# Name of map
map_name = "./simVRPTW.html"
map.save(map_name)

In [168]:
# Add marker
destination = df_locations.drop(index=0)

for _,row in destination.iterrows():
    folium.Circle(
        location=[row['lat'],row['lon']],
        radius=10000,  # Radius in pixels
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip=row['name'],
    ).add_to(map)

""" folium.Marker(
    location=[df_locations.iloc[0]['lat'],df_locations.iloc[0]['lon']],
    popup=df_locations.iloc[0]['name'],
    icon=folium.Icon(color='lightred') 
    ).add_to(map) """

folium.Circle(
    location=[df_locations.iloc[0]['lat'],df_locations.iloc[0]['lon']],
    radius=10000,  # Radius in pixels
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.6,
    tooltip=df_locations.iloc[0]['name']
).add_to(map)

map.save(map_name)

In [169]:
# Add edge
for coordinate in coordinates.values():
    folium.PolyLine(
        locations=coordinate,
        color="#ff6f00",
        weight=1,
        tooltip="TSP"
    ).add_to(map)

map.save(map_name)